In [57]:
import pandas as pd
import nltk.data
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import numpy as np
import numpy as np
from nltk.tokenize import word_tokenize
import torch
import random
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
!pip install geopy
!pip install python-Levenshtein==0.12.0
!pip install jarowinkler

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\RSPRASAD\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [149]:
df = pd.read_excel('Input file.xlsx')

In [158]:
df2 = df.loc[:,['Account Name','Billing Country','Billing State/Province','Billing City','Billing Street','Billing Zip/Postal Code']]

In [165]:

df2 = df.loc[:,['Account Name','Billing Country','Billing State/Province','Billing City','Billing Street',\
                'Billing Zip/Postal Code']]
df_count= pd.DataFrame()

df_count['Entity'] = ''
df_count['Percent'] = ''
df_count['Number'] = ''

num_records = len(df2)

df2 = df2.iloc[0:num_records,:]
df2 = df2.astype(str).apply(lambda x:x.str.lower())
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
count = 0
flag = 0
stop_words = []
stop_words = set(stop_words)

df2['Account ID'] = df['Account ID']
def remove_stop_words(text):
    global df_count
    global flag
    global stop_words
  
    global count
    text2 = ''
    if(len(df_count)==0):
        stop_words = set(stopwords.words('english'))
  
    else:
  
        if(flag ==0):
            for count in range(len(df_count)):
                if(df_count.loc[count, 'Number']<100):
                    flag = 1
                    break
        
        
        stop_words = set(df_count.loc[0:count-1,'Entity'])
    
    
        
    word_tokens = word_tokenize(text)
 
    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
            text2 += w + ' '

    text2 = text2.strip()
    return text2

df3 = df2.copy(deep = True)
df2['Account Name'] = df2['Account Name'].apply(remove_stop_words)
df2['Billing Street'] = df2['Billing Street'].apply(remove_stop_words)
df2 = df2.astype(str).replace("[\'\".,()*+&\/\-\\\+\!\%:;?]","")
#df2=df3

print("Stop words (1st time) are...", stop_words)
df_count['Entity'] = df2['Account Name'].str.split(expand=True).stack().value_counts().index
df_count['Number'] = list(df2['Account Name'].str.split(expand=True).stack().value_counts().values)
sum1 = df2['Account Name'].str.split(expand=True).stack().value_counts().sum()
df_count['Percent'] = list((df2['Account Name'].str.split(expand=True).stack().value_counts()/sum1).values)


df3['Account Name'] = df3['Account Name'].apply(remove_stop_words)
df3['Billing Street'] = df3['Billing Street'].apply(remove_stop_words)
df3 = df3.astype(str).replace("[\'\".,()*+&\/\-\\\+\!\%:;?]","")
print("Stop words (2nd time) are...", stop_words)
#df2=df3

Stop words (1st time) are... {'my', 'what', 'hasn', 'any', "couldn't", 'few', 'for', 'their', 'd', 'its', 'should', 'has', 'isn', 'been', "shan't", 'yours', 'more', "aren't", 'her', 'doing', 'they', 'before', 'when', 'of', 'own', 've', "wouldn't", "that'll", 'by', 'against', 'how', "haven't", 'ours', "you'll", 'your', 'then', 'no', "don't", 'where', 'once', 'am', "should've", 'why', "weren't", "didn't", 'both', 'after', 'myself', 'mustn', 'does', 'whom', "you've", 'each', 'so', 'hers', 'same', 'yourself', 'herself', "wasn't", 'him', 'through', 'now', 'very', 'ain', 'off', "hasn't", 'are', 'that', 'further', 'ma', 'll', 'yourselves', 'those', 'until', 'it', 'having', 'other', 'but', 'as', 'won', 'which', 's', 'some', 'while', 'them', 'couldn', 'his', 'and', 'into', 'above', 'doesn', 'himself', 'such', 'a', 'most', 'an', 'weren', 'from', 'needn', 'did', 'didn', 'who', 'have', 'were', 'about', 'i', 'this', "it's", 'there', 'be', 'or', 'mightn', "doesn't", 'theirs', 'during', 'wasn', "migh

In [167]:
df_lookup = df.loc[:, ['Account ID', 'Account Name']]
df_lookup.head()
df2 = pd.merge(df2,df_lookup, on = 'Account ID', how = 'left')
df2.rename(columns = {'Account Name_x':'Account Name'}, inplace = True)

In [169]:
stop_words2 = []
for item in stop_words:
    if len(item)>1:
        stop_words2.append(item)

stop_words2 = set(stop_words)
print(stop_words2)
stop_words = stop_words2

{'arizona', 'co', 'air', 'pacific', 'pfizer', 'thermo', 'minnesota', 'technologies', 'county', 'industries', 'san', 'life', ',', 'chemical', 'community', 'administration', 'bioscience', 'maryland', 'food', 'boston', 'american', 'general', 'group', ')', 'river', 'national', 'valley', 'central', 'los', 'international', 'illinois', 'virginia', 'wisconsin', 'johns', 'blood', 'diagnostics', 'cancer', 'institute', 'global', 'alabama', 'school', 'materials', 'pharmaceuticals', 'us', 'southern', 'california', 'canada', 'memorial', 'consulting', 'therapeutics', 'industrial', 'childrens', 'howard', 'city', 'health', 'south', 'agriculture', "'s", 'new', 'centre', 'fisher', '&', 'associates', 'west', 'quest', 'ohio', 'lab', 'scientific', 'columbia', 'carolina', 'pennsylvania', 'medical', 'systems', 'inc', 'care', 'vwr', 'ltd', 'hughes', 'st.', 'army', 'environmental', 'georgia', 'veterans', 'products', 'manufacturing', 'massachusetts', 'power', 'texas', 'service', 'children', 'system', '-', 'corp'

In [ ]:
import pandas as pd
import Levenshtein

from jarowinkler import *

df3 = df2
len_df3 = len(df3)
score = 0
score2= 0
index = 0
df3['Address_All'] = ''

for j in range(0, len(df3)):
    df3.loc[j,'Address_All'] = str(df3.loc[j,'Billing Country']) + ' '+ str(df3.loc[j,'Billing State/Province'])\
+ ' '+  str(df3.loc[j,'Billing City']) + ' '+str(df3.loc[j,'Billing Street'])\
+ ' '+ str(df3.loc[j,'Billing Zip/Postal Code'])

df_accounts = pd.DataFrame()
df_accounts['Id1'] = ''
df_accounts['Name1'] = ''
df_accounts['Id2'] = ''
df_accounts['Name2'] = ''
df_accounts['Original Name1'] = ''
df_accounts['Original Name2'] = ''
'''df_accounts2['Name1'] = ''
df_accounts2['Name2'] = '''''


for i in range(0, len_df3):
    print("I am in account number..", i)
    for j in range( i+1, len_df3):
        '''df_accounts2.loc[index, 'Name1'] = df3.loc[i,'Account Name']
        df_accounts2.loc[index, 'Name2'] = df3.loc[j,'Account Name']'''
        temp1 = df3.loc[i, 'Account Name']
        temp2 = df3.loc[j,'Account Name']
        for item in stop_words:
            if(any(item == word for word in temp1.split()) and any(item == word for word in temp2.split())):
                temp1 = temp1.replace(item, ' ').strip()
                temp2 = temp2.replace(item, ' ').strip()
                #print(f"temp1 is {temp1} and temp2 is {temp2} and item is {item}")
                
        score  =  jaro_similarity(temp1, temp2)
        if(score > 0.91 and df3.loc[i, 'Account Name'] !='' and df3.loc[j,'Account Name']!= ''):
            #print(f"temp1 is {temp1} and temp2 is {temp2}")
            #print(f"original accounts are {df3.loc[i, 'Account Name_y']} and {df3.loc[j, 'Account Name_y']}")
            df_accounts.loc[index, 'Id1'] = df3.loc[i, 'Account ID']
            df_accounts.loc[index, 'Id2'] = df3.loc[j, 'Account ID']
            df_accounts.loc[index, 'Name1'] = temp1
            df_accounts.loc[index, 'Name2'] = temp2
            df_accounts.loc[index, 'Original Name1'] = df3.loc[i, 'Account Name_y']
            df_accounts.loc[index, 'Original Name2'] = df3.loc[j, 'Account Name_y']
            df_accounts.loc[index, 'Address1'] = df3.loc[i, 'Address_All']
            df_accounts.loc[index, 'Address2'] = df3.loc[j, 'Address_All']

            df_accounts.loc[index, 'Similarity_Name'] = score
            score2  =  jaro_similarity(df3.loc[i, 'Address_All'], df3.loc[j,'Address_All'])
            df_accounts.loc[index, 'Similarity_Address'] = score2
        index +=1



I am in account number.. 0
I am in account number.. 1
I am in account number.. 2
I am in account number.. 3
I am in account number.. 4
I am in account number.. 5
I am in account number.. 6
I am in account number.. 7
I am in account number.. 8
I am in account number.. 9
I am in account number.. 10
I am in account number.. 11
I am in account number.. 12
I am in account number.. 13
I am in account number.. 14
I am in account number.. 15
I am in account number.. 16
I am in account number.. 17
I am in account number.. 18
I am in account number.. 19
I am in account number.. 20
I am in account number.. 21
I am in account number.. 22
I am in account number.. 23
I am in account number.. 24
I am in account number.. 25
I am in account number.. 26
I am in account number.. 27
I am in account number.. 28
I am in account number.. 29
I am in account number.. 30
I am in account number.. 31
I am in account number.. 32
I am in account number.. 33
I am in account number.. 34
I am in account number.. 35
I 

I am in account number.. 287
I am in account number.. 288
I am in account number.. 289
I am in account number.. 290
I am in account number.. 291
I am in account number.. 292
I am in account number.. 293
I am in account number.. 294
I am in account number.. 295
I am in account number.. 296
I am in account number.. 297
I am in account number.. 298
I am in account number.. 299
I am in account number.. 300
I am in account number.. 301
I am in account number.. 302
I am in account number.. 303
I am in account number.. 304
I am in account number.. 305
I am in account number.. 306
I am in account number.. 307
I am in account number.. 308
I am in account number.. 309
I am in account number.. 310
I am in account number.. 311
I am in account number.. 312
I am in account number.. 313
I am in account number.. 314
I am in account number.. 315
I am in account number.. 316
I am in account number.. 317
I am in account number.. 318
I am in account number.. 319
I am in account number.. 320
I am in accoun

I am in account number.. 570
I am in account number.. 571
I am in account number.. 572
I am in account number.. 573
I am in account number.. 574
I am in account number.. 575
I am in account number.. 576
I am in account number.. 577
I am in account number.. 578
I am in account number.. 579
I am in account number.. 580
I am in account number.. 581
I am in account number.. 582
I am in account number.. 583
I am in account number.. 584
I am in account number.. 585
I am in account number.. 586
I am in account number.. 587
I am in account number.. 588
I am in account number.. 589
I am in account number.. 590
I am in account number.. 591
I am in account number.. 592
I am in account number.. 593
I am in account number.. 594
I am in account number.. 595
I am in account number.. 596
I am in account number.. 597
I am in account number.. 598
I am in account number.. 599
I am in account number.. 600
I am in account number.. 601
I am in account number.. 602
I am in account number.. 603
I am in accoun

I am in account number.. 853
I am in account number.. 854
I am in account number.. 855
I am in account number.. 856
I am in account number.. 857
I am in account number.. 858
I am in account number.. 859
I am in account number.. 860
I am in account number.. 861
I am in account number.. 862
I am in account number.. 863
I am in account number.. 864
I am in account number.. 865
I am in account number.. 866
I am in account number.. 867
I am in account number.. 868
I am in account number.. 869
I am in account number.. 870
I am in account number.. 871
I am in account number.. 872
I am in account number.. 873
I am in account number.. 874
I am in account number.. 875
I am in account number.. 876
I am in account number.. 877
I am in account number.. 878
I am in account number.. 879
I am in account number.. 880
I am in account number.. 881
I am in account number.. 882
I am in account number.. 883
I am in account number.. 884
I am in account number.. 885
I am in account number.. 886
I am in accoun

I am in account number.. 1131
I am in account number.. 1132
I am in account number.. 1133
I am in account number.. 1134
I am in account number.. 1135
I am in account number.. 1136
I am in account number.. 1137
I am in account number.. 1138
I am in account number.. 1139
I am in account number.. 1140
I am in account number.. 1141
I am in account number.. 1142
I am in account number.. 1143
I am in account number.. 1144
I am in account number.. 1145
I am in account number.. 1146
I am in account number.. 1147
I am in account number.. 1148
I am in account number.. 1149
I am in account number.. 1150
I am in account number.. 1151
I am in account number.. 1152
I am in account number.. 1153
I am in account number.. 1154
I am in account number.. 1155
I am in account number.. 1156
I am in account number.. 1157
I am in account number.. 1158
I am in account number.. 1159
I am in account number.. 1160
I am in account number.. 1161
I am in account number.. 1162
I am in account number.. 1163
I am in ac

...

I am in account number.. 15467
I am in account number.. 15468
I am in account number.. 15469
I am in account number.. 15470
I am in account number.. 15471
I am in account number.. 15472
I am in account number.. 15473
I am in account number.. 15474
I am in account number.. 15475
I am in account number.. 15476
I am in account number.. 15477
I am in account number.. 15478
I am in account number.. 15479
I am in account number.. 15480
I am in account number.. 15481
I am in account number.. 15482
I am in account number.. 15483
I am in account number.. 15484
I am in account number.. 15485
I am in account number.. 15486
I am in account number.. 15487
I am in account number.. 15488
I am in account number.. 15489
I am in account number.. 15490
I am in account number.. 15491
I am in account number.. 15492
I am in account number.. 15493
I am in account number.. 15494
I am in account number.. 15495
I am in account number.. 15496
I am in account number.. 15497
I am in account number.. 15498
I am in 

I am in account number.. 15732
I am in account number.. 15733
I am in account number.. 15734
I am in account number.. 15735
I am in account number.. 15736
I am in account number.. 15737
I am in account number.. 15738
I am in account number.. 15739
I am in account number.. 15740
I am in account number.. 15741
I am in account number.. 15742
I am in account number.. 15743
I am in account number.. 15744
I am in account number.. 15745
I am in account number.. 15746
I am in account number.. 15747
I am in account number.. 15748
I am in account number.. 15749
I am in account number.. 15750
I am in account number.. 15751
I am in account number.. 15752
I am in account number.. 15753
I am in account number.. 15754
I am in account number.. 15755
I am in account number.. 15756
I am in account number.. 15757
I am in account number.. 15758
I am in account number.. 15759
I am in account number.. 15760
I am in account number.. 15761
I am in account number.. 15762
I am in account number.. 15763
I am in 

I am in account number.. 15997
I am in account number.. 15998
I am in account number.. 15999
I am in account number.. 16000
I am in account number.. 16001
I am in account number.. 16002
I am in account number.. 16003
I am in account number.. 16004
I am in account number.. 16005
I am in account number.. 16006
I am in account number.. 16007
I am in account number.. 16008
I am in account number.. 16009
I am in account number.. 16010
I am in account number.. 16011
I am in account number.. 16012
I am in account number.. 16013
I am in account number.. 16014
I am in account number.. 16015
I am in account number.. 16016
I am in account number.. 16017
I am in account number.. 16018
I am in account number.. 16019
I am in account number.. 16020
I am in account number.. 16021
I am in account number.. 16022
I am in account number.. 16023
I am in account number.. 16024
I am in account number.. 16025
I am in account number.. 16026
I am in account number.. 16027
I am in account number.. 16028
I am in 

I am in account number.. 16262
I am in account number.. 16263
I am in account number.. 16264
I am in account number.. 16265
I am in account number.. 16266
I am in account number.. 16267
I am in account number.. 16268
I am in account number.. 16269
I am in account number.. 16270
I am in account number.. 16271
I am in account number.. 16272
I am in account number.. 16273
I am in account number.. 16274
I am in account number.. 16275
I am in account number.. 16276
I am in account number.. 16277
I am in account number.. 16278
I am in account number.. 16279
I am in account number.. 16280
I am in account number.. 16281
I am in account number.. 16282
I am in account number.. 16283
I am in account number.. 16284
I am in account number.. 16285
I am in account number.. 16286
I am in account number.. 16287
I am in account number.. 16288
I am in account number.. 16289
I am in account number.. 16290
I am in account number.. 16291
I am in account number.. 16292
I am in account number.. 16293
I am in 

I am in account number.. 16527
I am in account number.. 16528
I am in account number.. 16529
I am in account number.. 16530
I am in account number.. 16531
